```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 

# Demo 3 - Source PGSQL16 - Target PGSQL17

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.PostgreSQL , *-*"

Installed Packages Microsoft.DotNet.Interactive.PostgreSQL, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.postgresql\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query PostgreSQL databases. 
 This extension adds support for connecting to PostgreSql databases using the #!connect postgres magic command.

## Working on TPCH SF10 ORDERS (15M rows)

### PGSQL 16 to PG 17

In [2]:
// Connect to PostgreSQL target database
#!connect postgres "Host=localhost;Port=25432;Database=tpch;Username=FastUser;Password=FastPassword;CommandTimeout=1500" --kernel-name pgsql17

Kernel added: #!sql-pgsql17

## Copy data from postgresql 16 to postgresql 17 using dblink (to compare)

#### Target orders_3 with a clustered ColumnStoreIndex

In [3]:
TRUNCATE TABLE tpch_10.orders;
SELECT count(*) count_target_table_rows from tpch_10.orders;

ALTER TABLE tpch_10.orders SET UNLOGGED;

count_target_table_rows
0


In [ ]:
DROP FOREIGN TABLE IF EXISTS "pg16_orders";
DROP USER MAPPING if exists for "FastUser" server "pg16tpch";
DROP SERVER if exists "pg16tpch";

In [14]:
CREATE SERVER "pg16tpch" FOREIGN DATA WRAPPER postgres_fdw OPTIONS (host 'localhost', dbname 'tpch', port '15432');
CREATE USER MAPPING IF NOT EXISTS FOR "FastUser" SERVER pg16tpch OPTIONS (user 'FastUser', password 'FastPassword');

In [15]:
-- Create external table in pg17 (target) to read data from pg16 (source)
DROP FOREIGN TABLE IF EXISTS pg16_orders;
CREATE FOREIGN TABLE IF NOT EXISTS pg16_orders (
    o_orderkey integer,
    o_custkey integer,
    o_orderstatus char(1),
    o_totalprice numeric(15,2),
    o_orderdate date,
    o_orderpriority char(15),
    o_clerk char(15),
    o_shippriority integer,
    o_comment varchar(79)
) SERVER pg16tpch OPTIONS (schema_name 'tpch_10', table_name 'orders');

### Copy data from postgresql 16 to postgresql 17 using the external table ==>


In [ ]:
INSERT INTO tpch_10.orders
SELECT * FROM pg16_orders;

In [17]:
SELECT count(*) count_target_table_rows_after_insert from tpch_10.orders;

count_target_table_rows_after_insert
15000000


## Test FastTransfer with Method Ctid (postgresql only)

CTID is a system hidden column that stores the physical location (block number and tuple offset) of a row version within its table.

You can SELECT it and even filter data using this hidden column. FastTransfer use the Ctid column to select data by chunck. It also create a micro-postgresql-snapshot before openning several connections to the source in order to have a stable source for each threads that will extract a portion of the data. 

In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "orders" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -2 `
--runid "PG16_to_PG17"

2025-09-22T00:33:45.587 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- DEBUG -|- tpch.tpch_10.orders -|- Using log settings from FastTransfer_Settings.json
2025-09-22T00:33:45.596 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer – running in normal mode.
2025-09-22T00:33:45.615 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- License for Architecture & Performance validated successfully. License valid until 2026-09-05
2025-09-22T00:33:45.615 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Starting
2025-09-22T00:33:45.615 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T00:33:45.616 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-22T00:33:45.616 +02:00 -|- FastTransfer 

In [8]:
SELECT count(*) count_target_table_rows from tpch_10.orders;

count_target_table_rows
15000000


In [17]:
SELECT * FROM tpch_10.orders LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
24552903,1313752,F,305754.99,1992-03-05 00:00:00Z,3-MEDIUM,Clerk#000007605,0,ly regular requests within the furiously regul
24575648,978844,F,175154.42,1992-12-10 00:00:00Z,2-HIGH,Clerk#000001182,0,y across the requests. quickly special accounts cajole fluffily
24575651,624299,F,116155.58,1992-02-09 00:00:00Z,4-NOT SPECIFIED,Clerk#000009852,0,ironic packages. fluffily express courts are across the slyly ironic theodo
24575682,201691,F,116158.29,1992-06-14 00:00:00Z,4-NOT SPECIFIED,Clerk#000004253,0,t the blithely final pinto beans; furio
24575712,1435832,F,330511.13,1992-11-27 00:00:00Z,1-URGENT,Clerk#000008569,0,its. regular foxes boost quickly. slyly special dugouts sleep blithe


#### Target orders_heap : Heap Table

In [22]:
DROP TABLE IF EXISTS tpch_10.orders_heap;

CREATE TABLE tpch_10.orders_heap AS
SELECT * 
FROM tpch_10.orders
WHERE 1=0;

ALTER TABLE tpch_10.orders_heap SET UNLOGGED;

SELECT count(*) count_orders_heap from tpch_10.orders_heap;

count_orders_heap
0


In [23]:
INSERT INTO tpch_10.orders_heap
SELECT * FROM pg16_orders;


In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "orders" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders_heap" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -2 `
--runid "PG16_to_PG17_heap"

2025-09-22T12:00:58.654 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- The FastTransfer_Settings.json file does not exist. Using default settings. Console Only
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer - running in trial mode – will expires on 2025‑10‑23 (30 day(s) left).
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Starting
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer Version : 0.13.11.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- OS : Microsoft Windows 10.0.26100
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Process ID : 20452
2025-09-22T12:00:58.665 +02:00 -|-  -|- PG16_

In [12]:

SELECT count(*) count_rows_in_heap from tpch_10.orders_heap;

count_rows_in_heap
15000000


In [16]:
SELECT * from tpch_10.orders_heap LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
33660512,775268,F,57043.01,1994-09-30 00:00:00Z,4-NOT SPECIFIED,Clerk#000003931,0,"special, regular requests use."
33660514,1403410,F,69873.94,1994-04-22 00:00:00Z,4-NOT SPECIFIED,Clerk#000008318,0,final deposits abou
41281026,1211515,F,180962.41,1994-09-01 00:00:00Z,4-NOT SPECIFIED,Clerk#000005558,0,lithely final packages cajole daring
41281028,340883,F,243024.32,1994-04-23 00:00:00Z,1-URGENT,Clerk#000000631,0,nding packages. carefully bold pinto beans are furiously busily
33660550,192962,F,35781.16,1994-01-19 00:00:00Z,2-HIGH,Clerk#000004242,0,"final accounts sleep final, ironic pin"


#### Test using a bigger table : LineItem (60M Rows x 16 columns - 16GB) 

In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "lineitem" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "lineitem_heap_unlogged" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -2 `
--runid "PG16_to_PG17_60M_rowsx16_columns"

2025-09-22T12:01:32.394 +02:00 -|-  -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- The FastTransfer_Settings.json file does not exist. Using default settings. Console Only
2025-09-22T12:01:32.404 +02:00 -|-  -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer - running in trial mode – will expires on 2025‑10‑23 (30 day(s) left).
2025-09-22T12:01:32.405 +02:00 -|-  -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Starting
2025-09-22T12:01:32.405 +02:00 -|-  -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer Version : 0.13.11.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T12:01:32.405 +02:00 -|-  -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- OS : Microsoft Windows 10.0.26100
2025-09-22T12:01:32.405 +02:00 -|-  -|- P

#### Results with Ctid pgcopy ==> pgcopy (binary copy method) :

| PG16 ==> PG17 (Ctid method) 				| Elapsed Time (seconds)|Cells/sec 	|Parallel Dispatch Method|degree of parallelism
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: | :--: |
| FastTransfer Load LINEITEM (60M Rows x 16c columns) HEAP Target       | 16.6     			|57_831_325    | Ctid | 16 |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) HEAP Target 		| 3.4     				|39_705_882    | Ctid | 16 |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) Target with PK 		| 5.3     				|25_471_678    | Ctid | 16 |


#### Results with foreign table + insert (legacy method) :
| PG16 ==> PG17 (Foreign Table + Insert method) 	| Elapsed Time (seconds)|Cells/sec 	|Parallel Dispatch Method
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: |
| Postgresql fdw table  (15M Rows x  9c columns) Target with PK | 82.3     			|1_646_341     | N/A |
| Postgresql fdw table  (15M Rows x  9c columns) HEAP Target | 51.8     			|2_606_177     | N/A |



The speed of loads (in cells/sec) is higher with LINEITEM than with ORDERS because the LINEITEM table is bigger and the startup time take a smaller role/space in the total time than it take with ORDERS load.